<a href="https://colab.research.google.com/github/kathmandu777/Research_of_poker_guy/blob/main/img_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 画像分類(機械学習)

TensorFlow作成のコードを適宜改良しました。["chip_kick","move_to_pass","pass"]の三種類のクラスに画像が分類されます。<br>


*   参考:[TensorFlow rock_paper_scissors](https://www.tensorflow.org/datasets/catalog/rock_paper_scissors)
*   参考:[TensorFlow YouTube](https://www.youtube.com/watch?v=k15RnGEOz8o)




In [ ]:
#######################################
#####  学習する画像を読み込む    #######
#######################################

import os
import time

start = time.time()  # プログラムの実行時間を測定

#学習データのフォルダ位置
root_dir='/content/drive/MyDrive/ColabNotebooks/Kida-Research' #colabのGoogleDriveの位置
chip_kick_dir = os.path.join(root_dir,'train_data/chip_kick')
move_to_pass_dir = os.path.join(root_dir,'train_data/move_to_pass')
pass_dir = os.path.join(root_dir,'train_data/pass')

#画像枚数の表示
print('total training chip_kick images:', len(os.listdir(chip_kick_dir)))
print('total training move_to_pass images:', len(os.listdir(move_to_pass_dir)))
print('total training pass_dir images:', len(os.listdir(pass_dir)))


chip_kick_files = os.listdir(chip_kick_dir)  # 引数のディレクトリ位置にあるファイルをリスト形式で返す
print(chip_kick_files[:10])  # 先頭10枚のファイル名を表示

move_to_pass_files = os.listdir(move_to_pass_dir)
print(move_to_pass_files[:10])

pass_files = os.listdir(pass_dir)
print(pass_files[:10])



In [ ]:
#######################################
#####  学習する画像を任意枚数表示  #####
#######################################

%matplotlib inline

import matplotlib.pyplot as plt
import matplotlib.image as mpimg


pic_index = 2  # 表示する画像の枚数

next_chip_kick = [os.path.join(chip_kick_dir, fname) 
                for fname in chip_kick_files[pic_index-2:pic_index]]
next_move_to_pass = [os.path.join(move_to_pass_dir, fname) 
                for fname in move_to_pass_files[pic_index-2:pic_index]]
next_pass = [os.path.join(pass_dir, fname) 
                for fname in pass_files[pic_index-2:pic_index]]

for i, img_path in enumerate(next_chip_kick+next_move_to_pass+next_pass):
  print(img_path)
  img = mpimg.imread(img_path)
  plt.imshow(img)
  plt.axis('Off') # 軸の表示なし
  #plt.title(img_path)
  plt.show()

In [ ]:
#################################################
##   モデルの作成(おそらく。名称に自信がない)  ###
#################################################

import tensorflow as tf
import keras_preprocessing
from keras_preprocessing import image
from keras_preprocessing.image import ImageDataGenerator
from keras.utils import plot_model

import time
import datetime
from google.colab import files

#ImageDataGeneratorについて：データの前処理（せん断や回転など）を行うのに便利なモジュール
#ドキュメント=https://keras.io/ja/preprocessing/image/

TRAINING_DIR = "/content/drive/MyDrive/ColabNotebooks/Kida-Research/train_data" #学習する画像のあるフォルダ



#学習させる画像の変更度の定義(様々な画像に対応させるため)

#喜田さんの研究は必ず128*128の回転ナシの画像が与えられるためりスケールのみ行うように定義する
#上記のように最初は考えたが、過学習を防ぐために多少の変更を行うことはありかもしれない
training_datagen = ImageDataGenerator(
      rescale = 1./128  #画像サイズを0~1に収めるためにリスケールする
      
      # 以下のコメントアウトしてあるコードは消去してもよい。(マンズの勉強用に保存)
	    # rotation_range=0,#回転させない
      # width_shift_range=0,
      # height_shift_range=0,
      # shear_range=0,
      # zoom_range=0,
      # horizontal_flip=False,
      # fill_mode='nearest'#回転させたときに出てくる色のない部分を何色で埋めるかの定義(今回は発生させないため本来は定義不要(おそらく))
)

VALIDATION_DIR = "/content/drive/MyDrive/ColabNotebooks/Kida-Research/test_data" #テスト画像のあるフォルダ
validation_datagen = ImageDataGenerator(rescale = 1./128)


TRAIN_BATCHSIZE=50
train_generator = training_datagen.flow_from_directory( #training_datagenの定義を用いて写真を変更し保存
	TRAINING_DIR,#学習する画像のフォルダを指定
	target_size=(128,128),#画像データのサイズ
	class_mode='categorical',# "categorical"か"binary"か"sparse"か"input"か"other"か"None"のいずれか1つ．"categorical"は2次元のone-hotにエンコード化されたラベル
  batch_size=TRAIN_BATCHSIZE # batch_sizeはミニバッチ学習に使われるバッチのサイズ。
)
"""
補足「バッチサイズについて」
  学習データから設定したサイズごとにデータを取り出し、計算を行う
  値は任意に調整すれば良いが、テスト用データの枚数を割り切れる数にしないとおかしなことになる可能性があるので注意する。
  参考：https://qiita.com/kenta1984/items/bad75a37d552510e4682
"""
VAL_BATCHSIZE=50
validation_generator = validation_datagen.flow_from_directory(
	VALIDATION_DIR,
	target_size=(128,128),
	class_mode='categorical',
  batch_size=VAL_BATCHSIZE
)

#モデルの定義
#以下に関連するドキュメント
"""
Conv2Dについて（https://keras.io/ja/layers/convolutional/）
  kernel_sizeで指定した範囲の画像を見て、畳み込みを行っています。画像における畳み込み計算は、簡単にいうとフィルター処理です（このフィルターは、カーネルとも呼ばれます）。
  https://github.com/vdumoulin/conv_arithmeticのREADME.mdにあるgifを見るとわかりやすい

  Conv2D(filters, kernel_size, strides=(1, 1),...)
    filters: 整数で，出力空間の次元（つまり畳み込みにおける出力フィルタの数）
    kernel_size: 2次元の畳み込みウィンドウの幅と高さを指定します. 単一の整数の場合は正方形のカーネル.->カーネル?(フィルター?)のサイズを指定
    activation: 使用する活性化関数の名前(活性化関数一覧：https://keras.io/ja/activations/)
    input_shape: 128*128 RGB画像の時は(128, 128, 3)
"""

"""
MaxPooling2Dについて(https://keras.io/ja/layers/pooling/)
  pool_sizeの範囲を見て、その中で最も大きな値を次の層に渡します

  MaxPooling2D(pool_size=(2, 2), strides=None,...)
    pool_size: ダウンスケールする係数を決める 2つの整数のタプル（垂直，水平）． (2, 2) は画像をそれぞれの次元で半分にします． 1つの整数しか指定ないと，それぞれの次元に対して同じ値が用いられます．
"""

"""
Flattenについて（https://keras.io/ja/layers/core/）
  入力を平坦化する．バッチサイズに影響を与えません．
  ＝>2次元（2D）の特徴を1次元に引き延ばす操作
"""

#上の三つ(おそらく)はCNN（Convolutional Neural Network、畳み込みニューラルネットワーク）と呼ばれる操作

"""
Dropoutについて（https://keras.io/ja/layers/core/）
  入力にドロップアウトを適用する．過学習の防止に役立つ
  全結合の層とのつながりをランダムに切断してあげることで、過学習を防ぐ。rateはその切断する割合を示している。

  Dropout(rate, noise_shape=None, seed=None)
    rate： 0と1の間の浮動小数点数．入力ユニットをドロップする割合
"""

"""
Denseについて(https://keras.io/ja/layers/core/)
  通常の全結合ニューラルネットワークレイヤー．

  Dense(units, activation=None, use_bias=True,...)
      units：正の整数，出力空間の次元数＝ニューロンの数
      activation： 使用する活性化関数名 （活性化関数一覧：https://keras.io/ja/activations/） 何も指定しなければ，活性化は適用されない（すなわち，"線形"活性化： a(x) = x）
"""

model = tf.keras.models.Sequential([
    # Note the input shape is the desired size of the image 150x150 with 3 bytes color
    # ↑ おそらくinput_shape=(150, 150, 3)を適宜変更してという意味
    # This is the first convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu', input_shape=(128, 128, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    # The second convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The third convolution
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The fourth convolution
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # Flatten the results to feed into a DNN
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.5),
    # 512 neuron hidden layer
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(3, activation='softmax')
])

#summary()=作成したニューラルネットワークのモデル形状の概要を表示(上で定義したモデルの概要を表示(可視化)?)
model.summary()

"""
compileについて（https://keras.io/ja/models/model/）
  学習のためのモデルを設定

  compile(optimizer, loss=None, metrics=None,...)
    optimizer: 文字列（optimizer名）またはoptimizerのオブジェクト(詳細：https://keras.io/api/optimizers/)
    loss: 文字列（目的関数名）または目的関数（詳細：https://keras.io/api/losses/）
    metrics: 訓練時とテスト時にモデルにより評価される評価関数のリスト．一般的にはmetrics=['accuracy']を使うことになります．

"""
LOSS='categorical_crossentropy'
OPTIMIZER='rmsprop'
METRICS='accuracy'
model.compile(loss =LOSS , optimizer=OPTIMIZER, metrics=[METRICS])


"""
fitについて（https://keras.io/ja/models/model/）
  固定回数（データセットの反復）の試行でモデルを学習

  fit(epochs=1, verbose=1,  validation_data=None, validation_steps=None,steps_per_epoch=None,...)
    epochs: 訓練データ配列の反復回数を示す整数．<=>モデルを学習するエポック数（学習データ全体を何回繰り返し学習させるか）
    verbose: 進行状況の表示モード．0 = 表示なし，1 = プログレスバー，2 = 各試行毎に一行の出力．
    validation_data: 各試行の最後に損失とモデル評価関数を評価するために用いられる(x_val, y_val)のタプル
    steps_per_epoch: 終了した1エポックを宣言して次のエポックを始めるまでのステップ数の合計（サンプルのバッチ）．
    validation_steps: steps_per_epochを指定している場合のみ関係します．停止する前にバリデーションするステップの総数（サンプルのバッチ）．

お役立ち情報
  ModelCheckpoint:	各種タイミング（毎エポック終了時や、今まででlossが一番小さかったエポック終了時など）に、学習したモデルを保存できる。保存する名前を、そのエポック数や、lossの値にすることもできる
  EarlyStopping:	lossや正答率が変化しなくなったところで、学習を打ち切ることができる
  LearningRateScheduler:	学習率のスケジューリングができる
  CSVLogger:	各エポックの結果をCSVファイルに保存する
"""
EPOCHS=25
STEPS_PER_EPOCH=20
VALIDATION_DATA=validation_generator
VERBOSE=1
VALIDATION_STEPS=3
history = model.fit(train_generator, 
                    epochs=EPOCHS, 
                    steps_per_epoch=STEPS_PER_EPOCH, 
                    validation_data =VALIDATION_DATA , 
                    verbose = VERBOSE, 
                    validation_steps=VALIDATION_STEPS
                  )



"""
.h5形式のファイルについて(一応)
  Hierarchical Data Formatの略.階層化された形でデータを保存することができるファイル形式

  メリット
    csvよりも読み書きが早い(pickleよりは遅い)
    他の言語でも使える(pickleとは違う点)

"""
date_now=str(datetime.datetime.now().strftime('%Y%m%d-%H%M%S'))#現在時刻 (UTC or GMTのどちらか忘れた)
elapsed_time = time.time() - start #経過時間の算出
elapsed_time = datetime.timedelta(seconds=elapsed_time) #経過時間の換算 秒->時間or分

model.save("model_"+date_now+".h5")#モデルの保存(h5形式)
files.download("model_"+date_now+".h5")#ローカルにも保存

plot_model(model, to_file="model_"+date_now+".png",show_shapes=True)#モデルの保存(png形式)  expand_nestedはデフォルトではFalse
files.download("model_"+date_now+".png")#ローカルにも保存

In [40]:
#############################################
#今回の実行パラメータをtxtファイルで出力######
#############################################

with open('param_'+date_now+'.txt', 'w',newline="\r\n") as f:
  f.write("elapsed_time="+str(elapsed_time)+'\n\n')
  f.write('total training chip_kick images:'+str(len(os.listdir(chip_kick_dir)))+'\n')
  f.write('total training move_to_pass images:'+str(len(os.listdir(move_to_pass_dir)))+"\n")
  f.write('total training pass_dir images:'+str(len(os.listdir(pass_dir)))+"\n\n")
  f.write("train batch size = "+str(TRAIN_BATCHSIZE)+"\n")
  f.write("validation batch size = "+str(VAL_BATCHSIZE)+"\n\n")
  #f.write("model="+str(model)+"\n")
  model.summary(print_fn=lambda x: f.write(x + "\n"))
  f.write("\ncompile: loss="+LOSS+", optimizer="+OPTIMIZER+", metrics="+METRICS+"\n\n")
  f.write("fit: epochs="+str(EPOCHS)+", steps_per_epoch="+str(STEPS_PER_EPOCH)+", verbose="+str(VERBOSE)+", validation_steps="+str(VALIDATION_STEPS)+"\n")

files.download('param_'+date_now+'.txt')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [1]:
##################################################
###  train_dataとtest_dataの適合率のプロット  #####
##################################################

import matplotlib.pyplot as plt
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.plot(epochs,loss,'g',label="Training loss")
plt.plot(epochs,val_loss,'m',label="Validation loss")
plt.title('Training and validation accuracy and loss')
plt.legend(loc=0)
#plt.figure()


plt.savefig('plt_'+date_now+'.png')
files.download('plt_'+date_now+'.png')

plt.show()


NameError: ignored

In [ ]:
#######################################
## 指定の画像でモデルの精度を検査　#####
#######################################

import tensorflow as tf
import keras_preprocessing
import numpy as np
from google.colab import files
from keras.preprocessing import image
import os

try:#modelが定義されているか(90min,12h制限のリセットを受けていないか)
  model
except NameError:
  print("modelが定義されていません。\nモデルファイルの名前を入力してください")
  root_dir='/content/drive/MyDrive/ColabNotebooks/Kida-Research'
  model_list=os.listdir(os.path.join(root_dir,"model"))
  for i,file_name in enumerate(model_list):#modelフォルダにあるファイル名を列挙
    print("  "+str(i)+" : "+str(file_name))
  print("インデックスを入力 = ",end="")
  model=tf.keras.models.load_model(os.path.join(root_dir,"model/"+str(model_list[int(input())])))#modelファイル(.h5形式)を選択

class_name=["chip_kick","move_to_pass","pass"]

isContinue=True

while(isContinue):
  print("\n分類(識別)したい画像を選択してください(複数選択可)")
  uploaded = files.upload()#分類したい画像を選択
  print('\n確率配列=["chip_kick","move_to_pass","pass"]')
    
  for fn in uploaded.keys():
    # predicting images
    path = fn
    img = image.load_img(path, target_size=(128, 128))#128*128サイズで
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    images = np.vstack([x])
    try:
      classes = model.predict(images, batch_size=50)
    except NameError:
      classes = new_model.predict(images, batch_size=50)

    print(fn,end=" = ")
    print(class_name[np.argmax(classes)],end=" ")
    print(classes)
  
  print("\n分類(識別)を続けますか?(Y or n)",end=" ")
  isContinue=False if input()=="n" else True   # nだったら終了(nに重みを置いている)
print("終了")

# chip_kick画像自動生成
a_haichi.py

(喜田さんが作成したものを適宜編集)

In [15]:
#以下のスクリプトによって生成される画像の保存先を指定
#test_data <=> train_data　適宜入れ替え
save_dir_chip_kick='/content/drive/MyDrive/ColabNotebooks/Kida-Research/test_data/chip_kick'

num_creations=2000 # 画像の生成枚数

In [16]:
#a_haichi.pyのコピー
#->chip_kick

import cv2
import numpy as np
import random
import os
#!pip install pyautogui
#import pyautogui

print("start")

circSize = 5

for i in range(0, int(num_creations)):
    for j in range(0, 3):
        img = np.full((128, 128, 3), 255, dtype=np.uint8)  # 128×128 全面白
        circColor = (255, 0, 0)  # blue
        x1 = random.randint(5, 122)
        y1 = random.randint(120, 125)
        cv2.circle(img, (x1, y1), circSize, circColor, -1)

        circColor = (255, 0, 0)  # blue
        if x1 < 20:
            x2 = x1 + random.randint(0, 20)

        elif x1 >= 20 and x1 < 105:
            x2 = x1 + random.randint(-15, 20)

        else:
            x2 = x1 + random.randint(-20, 2)

        y2 = random.randint(2, 50)
        cv2.circle(img, (x2, y2), circSize, circColor, -1)

        circColor = (255, 0, 0)  # blue
        if x2 < 20:
            x3 = random.randint(35, 125)

        elif x2 >= 20 and x2 < 105:
            a = x2 - 15
            b = x2 + 15
            while True:
                x3 = random.randint(2, 125)
                if x3 < a or x3 > b:
                    break
        else:
            x3 = random.randint(2, 90)

        y3 = random.randint(2, 90)
        cv2.circle(img, (x3, y3), circSize, circColor, -1)

        circColor = (0, 255, 255)  # yellow
        x4 = x1 + random.randint(-3, 3)
        y4 = y1 - random.randint(18, 25)
        cv2.circle(img, (x4, y4), circSize, circColor, -1)

        circColor = (0, 255, 255)  # yellow
        if x4 < 107:
            x5 = x4 + random.randint(10, 20)

        elif x4 < 112:
            x5 = x4 + random.randint(10, 15)

        else:
            x5 = x4 - random.randint(30, 40)

        y5 = y4 + random.randint(-10, 5)
        cv2.circle(img, (x5, y5), circSize, circColor, -1)

        circColor = (0, 255, 255)  # yellow
        if x4 > 20:
            x6 = x4 - random.randint(10, 20)

        elif x4 > 15:
            x6 = x4 - random.randint(10, 15)

        else:
            x6 = x4 + random.randint(30, 40)

        y6 = y4 + random.randint(-10, 5)
        cv2.circle(img, (x6, y6), circSize, circColor, -1)

        circColor = (0, 0, 0)  # black
        x7 = x1 + random.randint(-5, 5)

        e = y4 + 7  # 　ボールの位置がy4ではだめ
        while True:
            y7 = y1 - random.randint(3, 7)
            if y7 > e:
                break

        cv2.circle(img, (x7, y7), 3, circColor, -1)

        #cv2.namedWindow("display")
        #cv2.imshow("display", img)
        #cv2.moveWindow("display", 100, 200)
        #cv2.waitKey(300)
        #screenshot = pyautogui.screenshot(region=(221, 228, 128, 128))
        #screenshot.save(os.path.join(save_dir_chip_kick,'ck_{0:04d}.png'.format(i)))
        #cv2.destroyWindow("display")
        cv2.imwrite(os.path.join(save_dir_chip_kick,'chip_kick_{0:04d}.png'.format(i)),img)

print("Done:chip_kick")

start
Done:chip_kick


# move_to_pass画像自動生成
b_haichi?.py

(喜田さんが作成したものを適宜編集)

In [17]:
#以下のスクリプトによって生成される画像の保存先を指定
#test_data <=> train_data　適宜入れ替え
save_dir_move_to_pass='/content/drive/MyDrive/ColabNotebooks/Kida-Research/train_data/move_to_pass'

num_creations=2000 #  画像の生成枚数 4で割り切れる数にする

In [18]:
#b_haichi1.py
import cv2
import numpy as np
import random
import os

print("start")

circSize = 5

for i in range(0,int(num_creations/4)):
    for j in range(0,3):
        img = np.full((128, 128, 3), 255, dtype = np.uint8) # 128×128 全面白
        circColor = (255, 0, 0)       #blue
        x1 = random.randint(43, 82)   #隣のマスと重ならないように（基準42~83）
        y1 = random.randint(42, 83)
        cv2.circle(img, (x1, y1), circSize, circColor, -1)

        circColor = (0, 255, 255)     #yellow
        x4 = x1 + random.randint(-10, 10)
        y4 = y1 - random.randint(18, 30)
        cv2.circle(img, (x4, y4), circSize, circColor, -1)

        circColor = (0, 255, 255)       #yellow
        x5 = random.randint(9, 40) 
        y5 = random.randint(2, 40)
        cv2.circle(img, (x5, y5), circSize, circColor, -1)

        circColor = (0, 255, 255)       #yellow
        x6 = random.randint(85, 118)
        y6 = random.randint(2, 40)
        cv2.circle(img, (x6, y6), circSize, circColor, -1)

        circColor = (255, 0, 0)     #blue
        a = x5 - 7
        x2 = random.randint(2, a)

        b = y5 + 5

        if y5 >= 2 and y5 < 37:
            y2 = random.randint(2, b)   # y5 + 5まで

        elif y6 >= 37 and y6 < 42:
            y3 = random.randint(2, 41)

        cv2.circle(img, (x2, y2), circSize, circColor, -1)

        circColor = (255, 0, 0)     #blue
        c = x6 + 7
        x3 = random.randint(c, 125)
        
        d = y6 + 5

        if y6 >= 2 and y6 < 37:
            y3 = random.randint(2, d)   # y6 + 5まで

        elif y6 >= 37 and y6 < 42:
            y3 = random.randint(2, 41)

        cv2.circle(img, (x3, y3), circSize, circColor, -1)

        circColor = (0, 0, 0)         #black
        x7 = x1 + random.randint(-5, 5)
        
        e = y4 + 7                    #　ボールの位置がy4ではだめ
        while True:
            y7 = y1 - random.randint(3, 7)
            if y7 > e:
                break

        cv2.circle(img, (x7, y7), 3, circColor, -1)

        cv2.imwrite(os.path.join(save_dir_move_to_pass,'move1_{0:04d}.png'.format(i)),img)


start


In [19]:
#b_haichi2.py
import cv2
import numpy as np
import random

circSize = 5

for i in range(0,int(num_creations/4)):
    for j in range(0,3):
        img = np.full((128, 128, 3), 255, dtype = np.uint8) # 128×128 全面白
        circColor = (255, 0, 0)       #blue
        x1 = random.randint(43, 81)   # 右のマスと重ならないように
        y1 = random.randint(42, 83)
        cv2.circle(img, (x1, y1), circSize, circColor, -1)

        circColor = (0, 255, 255)     #yellow
        x4 = x1 + random.randint(-10, 10)
        y4 = y1 - random.randint(18, 30)
        cv2.circle(img, (x4, y4), circSize, circColor, -1)

        circColor = (0, 255, 255)       #yellow
        x5 = random.randint(9, 40) 
        y5 = random.randint(2, 40)
        cv2.circle(img, (x5, y5), circSize, circColor, -1)

        circColor = (0, 255, 255)       #yellow
        x6 = random.randint(86, 118)
        y6 = random.randint(42, 83)
        cv2.circle(img, (x6, y6), circSize, circColor, -1)

        circColor = (255, 0, 0)     #blue
        a = x5 - 7
        x2 = random.randint(2, a)

        b = y5 + 5

        if y5 >= 2 and y5 < 37:
            y2 = random.randint(2, b)   # y6 + 5まで

        elif y5 >= 37 and y5 < 42:
            y2 = random.randint(2, 41)

        cv2.circle(img, (x2, y2), circSize, circColor, -1)

        circColor = (255, 0, 0)     #blue
        c = x6 + 7
        x3 = random.randint(c, 125)
        
        d = y6 + 5
        e = y6 - 5
        f = y6 - 10
        g = y6 + 10
        h = y6 - 15
        k = y6 + 15

        if y6 < 47:
            y3 = random.randint(42, d)  # y6 + 5まで

        elif y6 >= 47 and y6 < 52:
            y3 = random.randint(e, d)   # y6 - 5 から y6 + 5まで

        elif y6 >= 52 and y6 < 57:      # y6 - 10 から y6 + 10まで
            y3 = random.randint(f, g)

        elif y6 >= 57 and y6 < 69:      # y6 - 15 から y6 + 15まで
            y3 = random.randint(h, k)

        elif y6 >= 69 and y6 < 74:
            y3 = random.randint(f, g)   # y6 - 10 から y6 + 10まで

        elif y6 >= 74 and y6 < 79:
            y3 = random.randint(e, d)   # y6 - 5 から y6 + 5まで

        elif y6 >= 79 and y6 < 84:
            y3 = random.randint(e, 83)

        cv2.circle(img, (x3, y3), circSize, circColor, -1)

        circColor = (0, 0, 0)         #black
        x7 = x1 + random.randint(-5, 5)
        
        e = y4 + 7                    #　ボールの位置がy4ではだめ
        while True:
            y7 = y1 - random.randint(3, 7)
            if y7 > e:
                break  

        cv2.circle(img, (x7, y7), 3, circColor, -1)

        cv2.imwrite(os.path.join(save_dir_move_to_pass,'move2_{0:04d}.png'.format(i)),img)


In [20]:
#b_haichi3.py

import cv2
import numpy as np
import random

circSize = 5

for i in range(0,int(num_creations/4)):
    for j in range(0,3):
        img = np.full((128, 128, 3), 255, dtype = np.uint8) # 128×128 全面白
        circColor = (255, 0, 0)       #blue
        x1 = random.randint(44, 81)   #隣のマスと重ならないように（基準42~83）
        y1 = random.randint(42, 83)
        cv2.circle(img, (x1, y1), circSize, circColor, -1)

        circColor = (0, 255, 255)     #yellow
        x4 = x1 + random.randint(-10, 10)
        y4 = y1 - random.randint(17, 30)
        cv2.circle(img, (x4, y4), circSize, circColor, -1)

        circColor = (0, 255, 255)       #yellow
        x5 = random.randint(9, 39) 
        y5 = random.randint(42, 83)
        cv2.circle(img, (x5, y5), circSize, circColor, -1)

        circColor = (0, 255, 255)       #yellow
        x6 = random.randint(86, 118)
        y6 = random.randint(42, 83)
        cv2.circle(img, (x6, y6), circSize, circColor, -1)

        circColor = (255, 0, 0)     #blue
        a = x5 - 7
        x2 = random.randint(2, a)
        
        d1 = y5 + 5
        e1 = y5 - 5
        f1 = y5 - 10
        g1 = y5 + 10
        h1 = y5 - 15
        k1 = y5 + 15

        if y5 < 47:
            y2 = random.randint(42, d1)   # y5 + 5まで

        elif y5 >= 47 and y5 < 52:
            y2 = random.randint(e1, d1)   # y5 - 5 から y5 + 5まで

        elif y5 >= 52 and y5 < 57:        # y5 - 10 から y5 + 10まで
            y2 = random.randint(f1, g1)

        elif y5 >= 57 and y5 < 69:
            y2 = random.randint(h1,k1)    # y6 - 15 から y6 + 15まで

        elif y5 >= 69 and y5 < 74:
            y2 = random.randint(f1, g1)   # y5 - 10 から y5 + 10まで

        elif y5 >= 74 and y5 < 79:
            y2 = random.randint(e1, d1)   # y5 - 5 から y5 + 5まで

        elif y5 >= 79 and y5 < 84:
            y2 = random.randint(e1, 83)

        cv2.circle(img, (x2, y2), circSize, circColor, -1)

        circColor = (255, 0, 0)     #blue
        c = x6 + 7
        x3 = random.randint(c, 125)
        
        d = y6 + 5
        e = y6 - 5
        f = y6 - 10
        g = y6 + 10
        h = y6 - 15
        k = y6 + 15

        if y6 < 47:
            y3 = random.randint(42, d)  # y6 + 5まで

        elif y6 >= 47 and y6 < 52:
            y3 = random.randint(e, d)   # y6 - 5 から y6 + 5まで

        elif y6 >= 52 and y6 < 57:      # y6 - 10 から y6 + 10まで
            y3 = random.randint(f, g)

        elif y6 >= 57 and y6 < 69:      # y6 - 15 から y6 + 15まで
            y3 = random.randint(h, k)

        elif y6 >= 69 and y6 < 74:
            y3 = random.randint(f, g)   # y6 - 10 から y6 + 10まで

        elif y6 >= 74 and y6 < 79:
            y3 = random.randint(e, d)   # y6 - 5 から y6 + 5まで

        elif y6 >= 79 and y6 < 84:
            y3 = random.randint(e, 83)

        cv2.circle(img, (x3, y3), circSize, circColor, -1)

        circColor = (0, 0, 0)         #black
        x7 = x1 + random.randint(-5, 5)
        
        e = y4 + 7                    #　ボールの位置がy4ではだめ
        while True:
            y7 = y1 - random.randint(3, 7)
            if y7 > e:
                break  

        cv2.circle(img, (x7, y7), 3, circColor, -1)

        cv2.imwrite(os.path.join(save_dir_move_to_pass,'move3_{0:04d}.png'.format(i)),img)


In [21]:
#b_haichi4.pyのコピー
#->move_to_pass

import cv2
import numpy as np
import random

circSize = 5

for i in range(0,int(num_creations/4)):
    for j in range(0,3):
        img = np.full((128, 128, 3), 255, dtype = np.uint8) # 128×128 全面白
        circColor = (255, 0, 0)       #blue
        x1 = random.randint(44, 82)
        y1 = random.randint(42, 83)
        cv2.circle(img, (x1, y1), circSize, circColor, -1)

        circColor = (0, 255, 255)     #yellow
        x4 = x1 + random.randint(-10, 10)
        y4 = y1 - random.randint(18, 30)
        cv2.circle(img, (x4, y4), circSize, circColor, -1)

        circColor = (0, 255, 255)       #yellow
        x5 = random.randint(9, 39) 
        y5 = random.randint(42, 83)
        cv2.circle(img, (x5, y5), circSize, circColor, -1)

        circColor = (0, 255, 255)       #yellow
        x6 = random.randint(85, 118)
        y6 = random.randint(2, 40)
        cv2.circle(img, (x6, y6), circSize, circColor, -1)

        circColor = (255, 0, 0)     #blue
        a = x5 - 7
        x2 = random.randint(2, a)
        
        d = y5 + 5
        e = y5 - 5
        f = y5 - 10
        g = y5 + 10
        h = y5 - 15
        k = y5 + 15
        
        if y5 < 47:
            y2 = random.randint(42, d)  # y5 + 5まで

        elif y5 >= 47 and y5 < 52:
            y2 = random.randint(e, d)   # y5 - 5 から y5 + 5まで

        elif y5 >= 52 and y5 < 57:      # y5 - 10 から y5 + 10まで
            y2 = random.randint(f, g)

        elif y5 >= 57 and y5 < 69:      # y5 - 15 から y5 + 15まで
            y2 = random.randint(h, k)

        elif y5 >= 69 and y5 < 74:
            y2 = random.randint(f, g)   # y5 - 10 から y5 + 10まで

        elif y5 >= 74 and y5 < 79:
            y2 = random.randint(e, d)   # y5 - 5 から y5 + 5まで

        elif y5 >= 79 and y5 < 84:
            y2 = random.randint(e, 83)

        
        cv2.circle(img, (x2, y2), circSize, circColor, -1)

        circColor = (255, 0, 0)     #blue
        c = x6 + 7
        x3 = random.randint(c, 125)

        d1 = y6 + 5

        if y6 >= 2 and y6 < 37:
            y3 = random.randint(2, d1)   # y6 + 5まで

        elif y6 >= 37 and y6 < 42:
            y3 = random.randint(2, 41)

        cv2.circle(img, (x3, y3), circSize, circColor, -1)

        circColor = (0, 0, 0)         #black
        x7 = x1 + random.randint(-5, 5)
        
        e = y4 + 7                    #　ボールの位置がy4ではだめ
        while True:
            y7 = y1 - random.randint(3, 7)
            if y7 > e:
                break  

        cv2.circle(img, (x7, y7), 3, circColor, -1)

        cv2.imwrite(os.path.join(save_dir_move_to_pass,'move4_{0:04d}.png'.format(i)),img)

print("Done:move_to_pass")

Done:move_to_pass


# pass画像自動生成
c_haichi?.py

(喜田さんが作成したものを適宜編集)

In [22]:
#以下のスクリプトによって生成される画像の保存先を指定
#test_data <=> train_data　適宜入れ替え
save_dir_pass='/content/drive/MyDrive/ColabNotebooks/Kida-Research/train_data/pass'

num_creations=2000 # 画像の生成枚数(5で割り切れる数にする)

In [23]:
#c_haichi1.py

import cv2
import numpy as np
import random
import os

print("start")

circSize = 5

for i in range(0,int(num_creations/5)):
    for j in range(0,3):
        img = np.full((128, 128, 3), 255, dtype = np.uint8) # 128×128 全面白
        circColor = (255, 0, 0)       #blue
        x1 = random.randint(44, 81)   #隣のマスと重ならないように（基準42~83）
        y1 = random.randint(42, 83)
        cv2.circle(img, (x1, y1), circSize, circColor, -1)

        circColor = (0, 255, 255)     #yellow
        while True:
            x4 = x1 + random.randint(-10, 10)
            if x4 < 80:
                break

        y4 = y1 - random.randint(15, 25)
        cv2.circle(img, (x4, y4), circSize, circColor, -1)

        circColor = (0, 255, 255)       #yellow
        x5 = random.randint(9, 39) 
        y5 = random.randint(42, 83)
        cv2.circle(img, (x5, y5), circSize, circColor, -1)

        circColor = (0, 255, 255)       #yellow
        x6 = random.randint(86, 118)
        y6 = random.randint(42, 83)
        cv2.circle(img, (x6, y6), circSize, circColor, -1)

        circColor = (255, 0, 0)     #blue
        a = x5 - 7
        x2 = random.randint(2, a)
        
        d1 = y5 + 5
        e1 = y5 - 5
        f1 = y5 - 10
        g1 = y5 + 10
        h1 = y5 - 15
        k1 = y5 + 15

        if y5 < 47:
            y2 = random.randint(42, d1)   # y5 + 5まで

        elif y5 >= 47 and y5 < 52:
            y2 = random.randint(e1, d1)   # y5 - 5 から y5 + 5まで

        elif y5 >= 52 and y5 < 57:        # y5 - 10 から y5 + 10まで
            y2 = random.randint(f1, g1)

        elif y5 >= 57 and y5 < 69:
            y2 = random.randint(h1,k1)    # y6 - 15 から y6 + 15まで

        elif y5 >= 69 and y5 < 74:
            y2 = random.randint(f1, g1)   # y5 - 10 から y5 + 10まで

        elif y5 >= 74 and y5 < 79:
            y2 = random.randint(e1, d1)   # y5 - 5 から y5 + 5まで

        elif y5 >= 79 and y5 < 84:
            y2 = random.randint(e1, 83)

        cv2.circle(img, (x2, y2), circSize, circColor, -1)

        circColor = (255, 0, 0)     #blue
        x3 = random.randint(85, 125)
        
        y3 = random.randint(2, 37)  #下のマスとかぶらないように
        cv2.circle(img, (x3, y3), circSize, circColor, -1)

        circColor = (0, 0, 0)        #black
        x7 = x1 + random.randint(-5, 5)
        
        e = y4 + 7                    #　ボールの位置がy4ではだめ
        while True:
            y7 = y1 - random.randint(3, 7)
            if y7 > e:
                break  

        cv2.circle(img, (x7, y7), 3, circColor, -1)

        cv2.imwrite(os.path.join(save_dir_pass,'pass1_{0:04d}.png'.format(i)),img)


start


In [24]:
#c_haichi2.py

import cv2
import numpy as np
import random

circSize = 5

for i in range(0,int(num_creations/5)):
    for j in range(0,3):
        img = np.full((128, 128, 3), 255, dtype = np.uint8) # 128×128 全面白
        circColor = (255, 0, 0)       #blue
        x1 = random.randint(44, 81)   #隣のマスと重ならないように（基準42~83）
        y1 = random.randint(42, 83)
        cv2.circle(img, (x1, y1), circSize, circColor, -1)

        circColor = (0, 255, 255)     #yellow
        x4 = x1 + random.randint(-10, 10)
        y4 = y1 - random.randint(17, 30)
        cv2.circle(img, (x4, y4), circSize, circColor, -1)

        circColor = (0, 255, 255)       #yellow
        x5 = random.randint(9, 39) 
        y5 = random.randint(42, 83)
        cv2.circle(img, (x5, y5), circSize, circColor, -1)

        circColor = (0, 255, 255)       #yellow
        x6 = random.randint(86, 118)
        y6 = random.randint(42, 83)
        cv2.circle(img, (x6, y6), circSize, circColor, -1)

        circColor = (255, 0, 0)     #blue
        x2 = random.randint(2, 39)
        
        y2 = random.randint(2, 37)

        cv2.circle(img, (x2, y2), circSize, circColor, -1)

        circColor = (255, 0, 0)     #blue
        c = x6 + 7
        x3 = random.randint(c, 125)
        
        d = y6 + 5
        e = y6 - 5
        f = y6 - 10
        g = y6 + 10
        h = y6 - 15
        k = y6 + 15

        if y6 < 47:
            y3 = random.randint(42, d)  # y6 + 5まで

        elif y6 >= 47 and y6 < 52:
            y3 = random.randint(e, d)   # y6 - 5 から y6 + 5まで

        elif y6 >= 52 and y6 < 57:      # y6 - 10 から y6 + 10まで
            y3 = random.randint(f, g)

        elif y6 >= 57 and y6 < 69:      # y6 - 15 から y6 + 15まで
            y3 = random.randint(h, k)

        elif y6 >= 69 and y6 < 74:
            y3 = random.randint(f, g)   # y6 - 10 から y6 + 10まで

        elif y6 >= 74 and y6 < 79:
            y3 = random.randint(e, d)   # y6 - 5 から y6 + 5まで

        elif y6 >= 79 and y6 < 84:
            y3 = random.randint(e, 83)

        cv2.circle(img, (x3, y3), circSize, circColor, -1)

        circColor = (0, 0, 0)         #black
        x7 = x1 + random.randint(-5, 5)
        
        e = y4 + 7                    #　ボールの位置がy4ではだめ
        while True:
            y7 = y1 - random.randint(3, 7)
            if y7 > e:
                break  

        cv2.circle(img, (x7, y7), 3, circColor, -1)

        cv2.imwrite(os.path.join(save_dir_pass,'pass2_{0:04d}.png'.format(i+20)),img)


In [25]:
#c_haichi3.py

import cv2
import numpy as np
import random

circSize = 5

for i in range(0,int(num_creations/5)):
    for j in range(0,3):
        img = np.full((128, 128, 3), 255, dtype = np.uint8) # 128×128 全面白
        circColor = (255, 0, 0)       #blue
        x1 = random.randint(43, 82)   #隣のマスと重ならないように（基準42~83）
        y1 = random.randint(42, 83)
        cv2.circle(img, (x1, y1), circSize, circColor, -1)

        circColor = (0, 255, 255)     #yellow
        while True:
            x4 = x1 + random.randint(-10, 10)
            if x4 < 80:
                break

        y4 = y1 - random.randint(18, 30)
        cv2.circle(img, (x4, y4), circSize, circColor, -1)

        circColor = (0, 255, 255)       #yellow
        x5 = random.randint(9, 40) 
        y5 = random.randint(2, 40)
        cv2.circle(img, (x5, y5), circSize, circColor, -1)

        circColor = (0, 255, 255)       #yellow
        x6 = random.randint(85, 125)
        y6 = random.randint(2, 40)
        cv2.circle(img, (x6, y6), circSize, circColor, -1)

        circColor = (255, 0, 0)     #blue
        a = x5 - 7
        x2 = random.randint(2, a)

        b = y5 + 5

        if y5 >= 2 and y5 < 37:
            y2 = random.randint(2, b)   # y5 + 5まで

        elif y6 >= 37 and y6 < 42:
            y2 = random.randint(2, 41)

        cv2.circle(img, (x2, y2), circSize, circColor, -1)

        circColor = (255, 0, 0)     #blue
        while True:
            x3 = random.randint(85, 125)
            c = x1 + 15
            if x3 > c:              #x1のロボと15以上離す
                break
        
        y3 = random.randint(43, 83)

        cv2.circle(img, (x3, y3), circSize, circColor, -1)

        circColor = (0, 0, 0)         #black
        x7 = x1 + random.randint(-5, 5)
        
        e = y4 + 7                    #　ボールの位置がy4ではだめ
        while True:
            y7 = y1 - random.randint(3, 7)
            if y7 > e:
                break

        cv2.circle(img, (x7, y7), 3, circColor, -1)

        cv2.imwrite(os.path.join(save_dir_pass,'pass3_{0:04d}.png'.format(i+40)),img)


In [26]:
#c_haichi4.py

import cv2
import numpy as np
import random

circSize = 5

for i in range(0,int(num_creations/5)):
    for j in range(0,3):
        img = np.full((128, 128, 3), 255, dtype = np.uint8) # 128×128 全面白
        circColor = (255, 0, 0)       #blue
        x1 = random.randint(44, 82)   #隣のマスと重ならないように（基準42~83）
        y1 = random.randint(42, 83)
        cv2.circle(img, (x1, y1), circSize, circColor, -1)

        circColor = (0, 255, 255)     #yellow
        while True:
            x4 = x1 + random.randint(-10, 10)
            if x4 > 43:
                break

        y4 = y1 - random.randint(18, 30)
        cv2.circle(img, (x4, y4), circSize, circColor, -1)

        circColor = (0, 255, 255)       #yellow
        x5 = random.randint(2, 40) 
        y5 = random.randint(2, 38)
        cv2.circle(img, (x5, y5), circSize, circColor, -1)

        circColor = (0, 255, 255)       #yellow
        x6 = random.randint(85, 118)
        y6 = random.randint(2, 40)
        cv2.circle(img, (x6, y6), circSize, circColor, -1)

        circColor = (255, 0, 0)     #blue
        while True:
            x2 = random.randint(2, 41)
            a = x1 - 15
            if x2 < a:
                break

        y2 = random.randint(43, 83)

        cv2.circle(img, (x2, y2), circSize, circColor, -1)

        circColor = (255, 0, 0)     #blue
        c = x6 + 7
        x3 = random.randint(c, 125)
        
        d = y6 + 5

        if y6 >= 2 and y6 < 37:
            y3 = random.randint(2, d)   # y6 + 5まで

        elif y6 >= 37 and y6 < 42:
            y3 = random.randint(2, 41)

        cv2.circle(img, (x3, y3), circSize, circColor, -1)

        circColor = (0, 0, 0)         #black
        x7 = x1 + random.randint(-5, 5)
        
        e = y4 + 7                    #　ボールの位置がy4ではだめ
        while True:
            y7 = y1 - random.randint(3, 7)
            if y7 > e:
                break

        cv2.circle(img, (x7, y7), 3, circColor, -1)

        cv2.imwrite(os.path.join(save_dir_pass,'pass4_{0:04d}.png'.format(i+60)),img)


In [27]:
#c_haichi5.py
#->pass
import cv2
import numpy as np
import random

circSize = 5

for i in range(0,int(num_creations/5)):
    for j in range(0,3):
        img = np.full((128, 128, 3), 255, dtype = np.uint8) # 128×128 全面白
        circColor = (255, 0, 0)       #blue
        x1 = random.randint(44, 82)   #隣のマスと重ならないように（基準42~83）
        y1 = random.randint(42, 83)
        cv2.circle(img, (x1, y1), circSize, circColor, -1)

        circColor = (0, 255, 255)     #yellow
        while True:
            x4 = x1 + random.randint(-10, 10)
            if x4 > 43:
                break

        y4 = y1 - random.randint(18, 30)
        cv2.circle(img, (x4, y4), circSize, circColor, -1)

        circColor = (0, 255, 255)       #yellow
        x5 = random.randint(2, 40) 
        y5 = random.randint(2, 38)
        cv2.circle(img, (x5, y5), circSize, circColor, -1)

        circColor = (0, 255, 255)       #yellow
        x6 = random.randint(85, 125)
        y6 = random.randint(2, 38)
        cv2.circle(img, (x6, y6), circSize, circColor, -1)

        circColor = (255, 0, 0)     #blue
        while True:
            x2 = random.randint(2, 41)
            a = x1 - 15
            if x2 < a:
                break

        y2 = random.randint(43, 83)

        cv2.circle(img, (x2, y2), circSize, circColor, -1)

        circColor = (255, 0, 0)     #blue
        while True:
            x3 = random.randint(85, 125)
            b = x1 + 15
            if x3 > b:              #x1のロボと15以上離す
                break

        y3 = random.randint(43, 83)

        cv2.circle(img, (x3, y3), circSize, circColor, -1)

        circColor = (0, 0, 0)         #black
        x7 = x1 + random.randint(-5, 5)
        
        e = y4 + 7                    #　ボールの位置がy4ではだめ
        while True:
            y7 = y1 - random.randint(3, 7)
            if y7 > e:
                break

        cv2.circle(img, (x7, y7), 3, circColor, -1)

        cv2.imwrite(os.path.join(save_dir_pass,'pass5_{0:04d}.png'.format(i+80)),img)

print("Done:pass")

Done:pass
